<a href="https://colab.research.google.com/github/Shubhankar9934/Shubhankar9934/blob/main/OptimizeCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [16]:
!pip install keras-tuner


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [18]:
print(tf.__version__)

2.8.2


In [19]:
fashion_mnist = keras.datasets.fashion_mnist

In [20]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [21]:
train_images = train_images/255.0
test_images = test_images/255.0

In [22]:
train_images[0].shape

(28, 28)

Reshape this train_images

In [23]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

Creating build_model function and we take hp as a parameter which is a hyperparameter.

In [24]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model
  

In [25]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [26]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [27]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.9079999923706055

Best val_accuracy So Far: 0.9163333177566528
Total elapsed time: 00h 03m 11s
INFO:tensorflow:Oracle triggered exit


In [28]:
model=tuner_search.get_best_models(num_models=1)[0]

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 128)       1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        102432    
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 112)               1734768   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 1,839,610
Trainable params: 1,839,610
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 10s 5ms/step - loss: 0.1438 - accuracy: 0.9454 - val_loss: 0.2654 - val_accuracy: 0.9120
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1062 - accuracy: 0.9601 - val_loss: 0.2833 - val_accuracy: 0.9125
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0749 - accuracy: 0.9721 - val_loss: 0.3163 - val_accuracy: 0.9137
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0572 - accuracy: 0.9785 - val_loss: 0.3370 - val_accuracy: 0.9162
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0422 - accuracy: 0.9843 - val_loss: 0.4306 - val_accuracy: 0.9118
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0364 - accuracy: 0.9872 - val_loss: 0.4860 - val_accuracy: 0.9140
Epoch 10/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0296 - accuracy: 0.9899 - val_loss: 0.5876 - val_accurac